In [62]:
import numpy as np
import pandas as pd

import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import os,csv,math,sys, joblib
import networkx as nx

import matplotlib.pyplot as plt
%matplotlib inline
import collections
import itertools
from sklearn.neural_network import MLPRegressor, MLPClassifier
import random
import copy
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
import sklearn.model_selection, sklearn.preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn import linear_model
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
#import pydot
from similaritymeasures import frechet_dist
import json
import tqdm
import matplotlib
seed = 99 # To reproduce the results
np.random.seed(seed)
torch.manual_seed(seed)
random.seed(seed)

In [63]:
data = pd.read_csv("sachs_10000.csv")
data.drop(data.columns[[0]], axis=1, inplace=True)
cols = data.columns
for i in cols:
    data[i] = data[i].map({'LOW': 0, 'AVG': 1, 'HIGH': 2})
mms = MinMaxScaler()
data[['PKC', 'PKA', 'Raf', 'Mek', 'Erk', 'Jnk', 'P38']] = mms.fit_transform(data[['PKC', 'PKA', 'Raf', 'Mek', 'Erk', 'Jnk', 'P38']])
data = data[['PKC', 'PKA', 'Raf', 'Mek', 'Erk', 'Jnk', 'P38', 'Akt']]

In [3]:
data.head()

,PKC,PKA,Raf,Mek,Erk,Jnk,P38,Akt
0,0.0,0.5,0.5,0.5,1.0,0.0,0.0,1
1,0.0,0.5,0.5,0.0,0.0,0.5,0.0,0
2,0.5,0.5,0.5,0.5,1.0,0.5,0.5,1
3,0.5,0.5,0.5,0.0,0.0,0.5,0.0,1
4,0.0,0.5,1.0,0.0,1.0,0.5,0.0,1


In [4]:
pkc_interventions = np.linspace(min(data['PKC']), max(data['PKC']), 3)
reg = MLPClassifier().fit(data['PKC'].values.reshape(-1,1), data['Akt'].values)
gt_pkc = reg.predict(pkc_interventions.reshape(-1,1))


pka_interventions = np.linspace(min(data['PKA']), max(data['PKA']), 3)
reg = MLPClassifier().fit(data[['PKA','PKC']].values, data['Akt'].values)
gt_pka = []
for alpha in np.linspace(min(data['PKA']), max(data['PKA']), 3):
    df1 = pd.DataFrame.copy(data[['PKA','PKC']])
    df1['PKA'] = alpha
    gt_pka.append(np.mean(reg.predict(df1.values)))
    

raf_interventions = np.linspace(min(data['Raf']), max(data['Raf']), 3)
reg = MLPClassifier().fit(data[['Raf','PKC', 'PKA']].values, data['Akt'].values)
gt_raf = []
for alpha in np.linspace(min(data['Raf']), max(data['Raf']), 3):
    df1 = pd.DataFrame.copy(data[['Raf','PKC', 'PKA']])
    df1['Raf'] = alpha
    gt_raf.append(np.mean(reg.predict(df1.values)))
    

reg = MLPClassifier().fit(data[['Mek', 'PKA']].values, data['Akt'].values)
gt_mek = []
for alpha in np.linspace(min(data['Mek']), max(data['Mek']), 3):
    df1 = pd.DataFrame.copy(data[['Mek', 'PKA']])
    df1['Mek'] = alpha
    gt_mek.append(np.mean(reg.predict(df1.values)))
    

reg = MLPClassifier().fit(data[['Erk', 'PKA']].values, data['Akt'].values)
gt_erk = []
for alpha in np.linspace(min(data['Erk']), max(data['Erk']), 3):
    df1 = pd.DataFrame.copy(data[['Erk', 'PKA']])
    df1['Erk'] = alpha
    gt_erk.append(np.mean(reg.predict(df1.values)))
    
gt_jnk = np.zeros_like(gt_erk)    
gt_p38 = np.zeros_like(gt_erk)
    
aces_gt=[]
aces_gt.append(gt_pkc-np.mean(gt_pkc))
aces_gt.append(gt_pka-np.mean(gt_pka))
aces_gt.append(gt_raf-np.mean(gt_raf))
aces_gt.append(gt_mek-np.mean(gt_mek))
aces_gt.append(gt_erk-np.mean(gt_erk))
aces_gt.append(gt_jnk-np.mean(gt_jnk))
aces_gt.append(gt_p38-np.mean(gt_p38))
np.save('./aces/aces_gt.npy',aces_gt,allow_pickle=True)

In [5]:
aces_gt

[array([0., 0., 0.]),
 array([ 0.5624, -0.2812, -0.2812]),
 array([-0.2254,  0.0988,  0.1266]),
 array([-0.1915,  0.    ,  0.1915]),
 array([-0.461 , -0.2695,  0.7305]),
 array([0., 0., 0.]),
 array([0., 0., 0.])]

In [64]:
def multi_acc(y_pred, y_test):
    y_pred_softmax = torch.log_softmax(y_pred, dim = 1)
    _, y_pred_tags = torch.max(y_pred_softmax, dim = 1)
    correct_pred = (y_pred_tags == y_test).float()
    acc = correct_pred.sum() / len(correct_pred)
    acc = torch.round(acc * 100)
    return acc

In [65]:
ensemble_size=5 # to get mean and std values

values = list(data.columns.values)
y = data[values[-1:]]
y = np.array(y, dtype='long')
X = data[values[:-1]]
X = np.array(X)

indices = np.random.choice(len(X), len(X), replace=False)
X_values = X[indices]
y_values = y[indices]

# Creating a Train and a Test Dataset
test_size = 1000
val_size = 1000

X_test = X_values[-test_size:]
X_trainval = X_values[:-test_size]
X_val = X_trainval[-val_size:]
X_train = X_trainval[:-val_size]

y_test = y_values[-test_size:]
y_trainval = y_values[:-test_size]
y_val = y_trainval[-val_size:]
y_train = y_trainval[:-val_size]

def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

# Interval / Epochs
interval = 5
epoch = 50
batch_size=64

In [66]:
class samp_network(nn.Module):
    def __init__(self, input_size=1):
        super().__init__()
        self.input_size=input_size
        self.fc1 = nn.Linear(self.input_size, 2)
        self.fc2 = nn.Linear(2, 2)
        self.fc3 = nn.Linear(2, 1)

    def forward(self,x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [67]:
class Model(nn.Module):
    def __init__(self, feature_dim, batch_size=64, device='cpu',sample_size=50000):
        super(Model, self).__init__()
        self.batchsize=batch_size

        self.causal_link_pkc_pka = samp_network()
        self.causal_link_pkc_pka_raf = samp_network(input_size=2)
        self.causal_link_pkc_pka_raf_mek = samp_network(input_size=3)
        self.causal_link_mek_pka_erk = samp_network(input_size=2)
        self.causal_link_pkc_pka_jnk = samp_network(input_size=2)
        self.causal_link_pkc_pka_p38 = samp_network(input_size=2)

        self.batchsize=batch_size
        self.first_layer = nn.Linear(7,6)
        self.second_layer = nn.Linear(6,5)
        self.third_layer = nn.Linear(5,5)
        self.fourth_layer = nn.Linear(5,4)
        self.regression_layer = nn.Linear(4, 3)
        self.sample_size = sample_size

    def forward(self, inp, phase='freeze', inde=0, alpha=0):
        if phase=='freeze':
            x = F.relu(self.first_layer(inp))
            x = F.relu(self.second_layer(x))
            x = F.relu(self.third_layer(x))
            x = F.relu(self.fourth_layer(x))
            prediction = self.regression_layer(x)
            return prediction

        elif phase=='train_dag':

            batch_size = inp.shape[0]
            pkc_sample = inp[:,0].reshape(batch_size, -1)

            pka_sample = self.causal_link_pkc_pka(pkc_sample)
            raf_sample = self.causal_link_pkc_pka_raf(torch.cat((pkc_sample, pka_sample), dim=1))
            mek_sample = self.causal_link_pkc_pka_raf_mek(torch.cat((pkc_sample, pka_sample, raf_sample), dim=1))
            erk_sample = self.causal_link_mek_pka_erk(torch.cat((mek_sample, pka_sample), dim=1))
            jnk_sample = self.causal_link_pkc_pka_jnk(torch.cat((pkc_sample, pka_sample), dim=1))
            p38_sample = self.causal_link_pkc_pka_p38(torch.cat((pkc_sample, pka_sample), dim=1))
            inp = torch.cat((pkc_sample, pka_sample, raf_sample, mek_sample, erk_sample, jnk_sample, p38_sample),dim=1)

            x = F.relu(self.first_layer(inp))
            x = F.relu(self.second_layer(x))
            x = F.relu(self.third_layer(x))
            x = F.relu(self.fourth_layer(x))
            prediction = self.regression_layer(x)
            
            return prediction, pka_sample, raf_sample, mek_sample, erk_sample, jnk_sample, p38_sample

        elif phase=='sample':
            if inde == 0:
                pkc_sample = torch.tensor([alpha]*self.sample_size, dtype=torch.float).reshape(self.sample_size,-1)
                pka_sample = self.causal_link_pkc_pka(torch.tensor(pkc_sample, dtype=torch.float))
                raf_sample = self.causal_link_pkc_pka_raf(torch.cat((torch.tensor(pkc_sample, dtype=torch.float), pka_sample), dim=1))
                mek_sample = self.causal_link_pkc_pka_raf_mek(torch.cat((torch.tensor(pkc_sample, dtype=torch.float), torch.tensor(pka_sample, dtype=torch.float), torch.tensor(raf_sample, dtype=torch.float)), dim=1))
                erk_sample = self.causal_link_mek_pka_erk(torch.cat((torch.tensor(mek_sample, dtype=torch.float), torch.tensor(pka_sample, dtype=torch.float)), dim=1))
                jnk_sample = self.causal_link_pkc_pka_jnk(torch.cat((torch.tensor(pkc_sample, dtype=torch.float), torch.tensor(pka_sample, dtype=torch.float)), dim=1))
                p38_sample = self.causal_link_pkc_pka_p38(torch.cat((torch.tensor(pkc_sample, dtype=torch.float), torch.tensor(pka_sample, dtype=torch.float)), dim=1))
                inp = torch.cat((pkc_sample, pka_sample, raf_sample, mek_sample, erk_sample, jnk_sample, p38_sample),dim=1)
                return inp

            elif inde == 1:
                pkc_sample = torch.tensor(inp[:,0].reshape(self.sample_size,-1), dtype=torch.float)
                pka_sample = torch.tensor([alpha]*self.sample_size, dtype=torch.float).reshape(self.sample_size,-1)
                raf_sample = self.causal_link_pkc_pka_raf(torch.cat((torch.tensor(pkc_sample, dtype=torch.float), pka_sample), dim=1))
                mek_sample = self.causal_link_pkc_pka_raf_mek(torch.cat((torch.tensor(pkc_sample, dtype=torch.float), torch.tensor(pka_sample, dtype=torch.float), torch.tensor(raf_sample, dtype=torch.float)), dim=1))
                erk_sample = self.causal_link_mek_pka_erk(torch.cat((torch.tensor(mek_sample, dtype=torch.float), torch.tensor(pka_sample, dtype=torch.float)), dim=1))
                jnk_sample = self.causal_link_pkc_pka_jnk(torch.cat((torch.tensor(pkc_sample, dtype=torch.float), torch.tensor(pka_sample, dtype=torch.float)), dim=1))
                p38_sample = self.causal_link_pkc_pka_p38(torch.cat((torch.tensor(pkc_sample, dtype=torch.float), torch.tensor(pka_sample, dtype=torch.float)), dim=1))
                inp = torch.cat((pkc_sample, pka_sample, raf_sample, mek_sample, erk_sample, jnk_sample, p38_sample),dim=1)
                return inp

            elif inde == 2:
                pkc_sample = torch.tensor(inp[:,0].reshape(self.sample_size,-1), dtype=torch.float)
                pka_sample = torch.tensor(inp[:,1].reshape(self.sample_size,-1), dtype=torch.float)
                raf_sample = torch.tensor([alpha]*self.sample_size, dtype=torch.float).reshape(self.sample_size,-1)
                mek_sample = self.causal_link_pkc_pka_raf_mek(torch.cat((torch.tensor(pkc_sample, dtype=torch.float), torch.tensor(pka_sample, dtype=torch.float), torch.tensor(raf_sample, dtype=torch.float)), dim=1))
                erk_sample = self.causal_link_mek_pka_erk(torch.cat((torch.tensor(mek_sample, dtype=torch.float), torch.tensor(pka_sample, dtype=torch.float)), dim=1))
                jnk_sample = self.causal_link_pkc_pka_jnk(torch.cat((torch.tensor(pkc_sample, dtype=torch.float), torch.tensor(pka_sample, dtype=torch.float)), dim=1))
                p38_sample = self.causal_link_pkc_pka_p38(torch.cat((torch.tensor(pkc_sample, dtype=torch.float), torch.tensor(pka_sample, dtype=torch.float)), dim=1))
                inp = torch.cat((pkc_sample, pka_sample, raf_sample, mek_sample, erk_sample, jnk_sample, p38_sample),dim=1)
                return inp

            elif inde == 3:
                pkc_sample = torch.tensor(inp[:,0].reshape(self.sample_size,-1), dtype=torch.float)
                pka_sample = torch.tensor(inp[:,1].reshape(self.sample_size,-1), dtype=torch.float)
                raf_sample = torch.tensor(inp[:,2].reshape(self.sample_size,-1), dtype=torch.float)
                mek_sample = torch.tensor([alpha]*self.sample_size, dtype=torch.float).reshape(self.sample_size,-1)
                erk_sample = self.causal_link_mek_pka_erk(torch.cat((torch.tensor(mek_sample, dtype=torch.float), torch.tensor(pka_sample, dtype=torch.float)), dim=1))
                jnk_sample = self.causal_link_pkc_pka_jnk(torch.cat((torch.tensor(pkc_sample, dtype=torch.float), torch.tensor(pka_sample, dtype=torch.float)), dim=1))
                p38_sample = self.causal_link_pkc_pka_p38(torch.cat((torch.tensor(pkc_sample, dtype=torch.float), torch.tensor(pka_sample, dtype=torch.float)), dim=1))
                inp = torch.cat((pkc_sample, pka_sample, raf_sample, mek_sample, erk_sample, jnk_sample, p38_sample),dim=1)
                return inp

            elif inde == 4:
                pkc_sample = torch.tensor(inp[:,0].reshape(self.sample_size,-1), dtype=torch.float)
                pka_sample = torch.tensor(inp[:,1].reshape(self.sample_size,-1), dtype=torch.float)
                raf_sample = torch.tensor(inp[:,2].reshape(self.sample_size,-1), dtype=torch.float)
                mek_sample = torch.tensor(inp[:,3].reshape(self.sample_size,-1), dtype=torch.float)
                erk_sample = torch.tensor([alpha]*self.sample_size, dtype=torch.float).reshape(self.sample_size,-1)
                jnk_sample = self.causal_link_pkc_pka_jnk(torch.cat((torch.tensor(pkc_sample, dtype=torch.float), torch.tensor(pka_sample, dtype=torch.float)), dim=1))
                p38_sample = self.causal_link_pkc_pka_p38(torch.cat((torch.tensor(pkc_sample, dtype=torch.float), torch.tensor(pka_sample, dtype=torch.float)), dim=1))
                inp = torch.cat((pkc_sample, pka_sample, raf_sample, mek_sample, erk_sample, jnk_sample, p38_sample),dim=1)
                return inp


            elif inde == 5:
                pkc_sample = torch.tensor(inp[:,0].reshape(self.sample_size,-1), dtype=torch.float)
                pka_sample = torch.tensor(inp[:,1].reshape(self.sample_size,-1), dtype=torch.float)
                raf_sample = torch.tensor(inp[:,2].reshape(self.sample_size,-1), dtype=torch.float)
                mek_sample = torch.tensor(inp[:,3].reshape(self.sample_size,-1), dtype=torch.float)
                erk_sample = torch.tensor(inp[:,4].reshape(self.sample_size,-1), dtype=torch.float)
                jnk_sample = torch.tensor([alpha]*self.sample_size, dtype=torch.float).reshape(self.sample_size,-1)
                p38_sample = self.causal_link_pkc_pka_p38(torch.cat((torch.tensor(pkc_sample, dtype=torch.float), torch.tensor(pka_sample, dtype=torch.float)), dim=1))
                inp = torch.cat((pkc_sample, pka_sample, raf_sample, mek_sample, erk_sample, jnk_sample, p38_sample),dim=1)
                return inp

            elif inde == 6:
                pkc_sample = torch.tensor(inp[:,0].reshape(self.sample_size,-1), dtype=torch.float)
                pka_sample = torch.tensor(inp[:,1].reshape(self.sample_size,-1), dtype=torch.float)
                raf_sample = torch.tensor(inp[:,2].reshape(self.sample_size,-1), dtype=torch.float)
                mek_sample = torch.tensor(inp[:,3].reshape(self.sample_size,-1), dtype=torch.float)
                erk_sample = torch.tensor(inp[:,4].reshape(self.sample_size,-1), dtype=torch.float)
                jnk_sample = torch.tensor(inp[:,5].reshape(self.sample_size,-1))
                p38_sample = torch.tensor([alpha]*self.sample_size, dtype=torch.float).reshape(self.sample_size,-1)
                inp = torch.cat((pkc_sample, pka_sample, raf_sample, mek_sample, erk_sample, jnk_sample, p38_sample),dim=1)
                return inp

In [68]:
class DataSet(Dataset):
    """
    Custom dataset to load data from csv file
    """
    def __init__(self, dataframe1, dataframe2):
        self.data_points = dataframe1
        self.targets = dataframe2

    def __len__(self):
        return len(self.data_points)

    def __getitem__(self, idx):
        input_point = torch.tensor(np.array(self.data_points[idx]), dtype=torch.float)
        target_point = torch.tensor(np.array(self.targets[idx]), dtype=torch.long)
        return input_point, target_point

# ERM

In [11]:
for ensemble in range(ensemble_size):
    loss_func = nn.CrossEntropyLoss()
    erm_model = Model(7,sample_size=len(data))
    optimizer = optim.Adam([{'params': erm_model.parameters()}], lr = 0.001, weight_decay=1e-4)

    for ep in range(0,epoch): 
        trainval = DataSet(X_train,y_train)
        train_loader = DataLoader(trainval, batch_size=batch_size)
        for input_data, target in train_loader:
            erm_model.zero_grad()
            output = erm_model(input_data)
            loss = loss_func(output,target.squeeze())
            loss.backward()
            optimizer.step()
        if ep%interval == 0:
            print(ep, interval)
            val = DataSet(X_val,y_val)
            val_loader = DataLoader(val, batch_size=1)
            acc_val = 0
            acc_test = 0
            
            for input_data, target in val_loader:
                output = erm_model(input_data)
                acc = multi_acc(output, target.unsqueeze(1))
                acc_val += acc

            print ('validation accuracy:', float(acc_val/len(val_loader)))
            testval = DataSet(X_test,y_test)
            test_loader = DataLoader(testval, batch_size=1)
            
            for input_data, target in test_loader:
                output = erm_model(input_data)
                acc = multi_acc(output, target.unsqueeze(1))
                acc_test += acc
                
            print('test accuracy:', float(acc_test/len(test_loader)))
            print()
    print("************")
    torch.save(erm_model, "models/erm_sachs_"+str(ensemble+1))

0 5
validation accuracy: 60.79999923706055
test accuracy: 59.599998474121094

5 5
validation accuracy: 78.5999984741211
test accuracy: 80.4000015258789

10 5
validation accuracy: 78.5
test accuracy: 80.5

15 5
validation accuracy: 78.69999694824219
test accuracy: 80.5999984741211

20 5
validation accuracy: 78.9000015258789
test accuracy: 81.0999984741211

25 5
validation accuracy: 79.30000305175781
test accuracy: 81.69999694824219

30 5
validation accuracy: 79.30000305175781
test accuracy: 81.69999694824219

35 5
validation accuracy: 79.30000305175781
test accuracy: 81.69999694824219

40 5
validation accuracy: 79.30000305175781
test accuracy: 81.69999694824219

45 5
validation accuracy: 79.30000305175781
test accuracy: 81.69999694824219

************
0 5
validation accuracy: 60.79999923706055
test accuracy: 59.599998474121094

5 5
validation accuracy: 60.79999923706055
test accuracy: 59.599998474121094

10 5
validation accuracy: 73.69999694824219
test accuracy: 73.30000305175781

15 5


# Causal Attributions

In [14]:
n_classes=3
num_feat=7
num_alpha=3

cov = np.cov(X_values, rowvar=False)
mean_vector = np.mean(X_values, axis=0)

aces_ca_total = []

for ensemble in range(ensemble_size):
    ace_ca_total = []
    model = torch.load("models/erm_sachs_"+str(ensemble+1))
    for output_index in range(0,1):
        for t in range(0,num_feat):
            expectation_do_x = []
            inp=copy.deepcopy(mean_vector)
            for x in np.linspace(0, 1, num_alpha):                
                inp[t] = x
                input_torchvar = autograd.Variable(torch.FloatTensor(inp), requires_grad=True)
                output=model(input_torchvar)
                o1=output.data.cpu()
                val=o1.numpy()[output_index]#first term in interventional expectation                                       

                grad_mask_gradient = torch.zeros(n_classes)
                grad_mask_gradient[output_index] = 1.0
                #calculating the hessian
                first_grads = torch.autograd.grad(output.cpu(), input_torchvar.cpu(), grad_outputs=grad_mask_gradient, retain_graph=True, create_graph=True)

                for dimension in range(0,num_feat):#Tr(Hessian*Covariance)
                    if dimension == t:
                        continue
                    temp_cov = copy.deepcopy(cov)
                    temp_cov[dimension][t] = 0.0#row,col in covariance corresponding to the intervened one made 0
                    grad_mask_hessian = torch.zeros(num_feat)
                    grad_mask_hessian[dimension] = 1.0

                    #calculating the hessian
                    hessian = torch.autograd.grad(first_grads, input_torchvar, grad_outputs=grad_mask_hessian, retain_graph=True, create_graph=False)
                    val += np.sum(0.5*hessian[0].data.numpy()*temp_cov[dimension])#adding second term in interventional expectation
                expectation_do_x.append(val)#append interventional expectation for given interventional value
            ace_ca_total.append(np.array(expectation_do_x) - np.mean(np.array(expectation_do_x)))
    aces_ca_total.append(ace_ca_total)
np.save('./aces/sachs_ca_total.npy',aces_ca_total,allow_pickle=True)

In [15]:
rmse_results = []
frechet_results = []

for ensemble in range(ensemble_size):
    rmse_results.append([rmse(aces_gt[0], aces_ca_total[ensemble][0]),
                         rmse(aces_gt[1], aces_ca_total[ensemble][1]),
                         rmse(aces_gt[2], aces_ca_total[ensemble][2]),
                         rmse(aces_gt[3], aces_ca_total[ensemble][3]),
                         rmse(aces_gt[4], aces_ca_total[ensemble][4]),
                         rmse(aces_gt[5], aces_ca_total[ensemble][5]),
                         rmse(aces_gt[6], aces_ca_total[ensemble][6])])
    
    frechet_results.append([frechet_dist(aces_gt[0].reshape(3,1), aces_ca_total[ensemble][0].reshape(3,1)),
                            frechet_dist(aces_gt[1].reshape(3,1), aces_ca_total[ensemble][1].reshape(3,1)),
                            frechet_dist(aces_gt[2].reshape(3,1), aces_ca_total[ensemble][2].reshape(3,1)),
                            frechet_dist(aces_gt[3].reshape(3,1), aces_ca_total[ensemble][3].reshape(3,1)),
                            frechet_dist(aces_gt[4].reshape(3,1), aces_ca_total[ensemble][4].reshape(3,1)),
                            frechet_dist(aces_gt[5].reshape(3,1), aces_ca_total[ensemble][5].reshape(3,1)),
                            frechet_dist(aces_gt[6].reshape(3,1), aces_ca_total[ensemble][6].reshape(3,1))])
    
rmse_results = np.array(rmse_results)
print("rmse mean: ", np.mean(rmse_results, axis=0))
print("rmse std: ", np.std(rmse_results, axis=0))
print("rmse all features mean: ", np.mean(np.mean(rmse_results, axis=0)))
print("rmse all features std: ", np.mean(np.std(rmse_results, axis=0)))

print("frechet mean: ", np.mean(frechet_results, axis=0))
print("frechet std: ", np.std(frechet_results, axis=0))
print("frechet all features mean: ", np.mean(np.mean(frechet_results, axis=0)))
print("frechet all features std: ", np.mean(np.std(frechet_results, axis=0)))

rmse mean:  [0.10176131 2.19258358 0.11545693 0.21651297 2.23200216 0.0742025
 0.09842272]
rmse std:  [0.09715836 0.9016019  0.05735296 0.13859572 0.63006274 0.0440616
 0.06976197]
rmse all features mean:  0.7187060241403819
rmse all features std:  0.27694218134075055
frechet mean:  [0.13223125 2.97188839 0.16011002 0.27970836 3.12497292 0.09462399
 0.12896724]
frechet std:  [0.12704554 1.14069842 0.08258818 0.18027261 0.90048447 0.05460573
 0.09408247]
frechet all features mean:  0.9846431686074393
frechet all features std:  0.36853963237833876


# CREDO

In [23]:
prior = {0:(lambda ii:ii), 1:(lambda ii:ii), 2:(lambda ii:ii),
        3:(lambda ii:ii), 4:(lambda ii:ii), 5:(lambda ii:0*ii),
        6:(lambda ii:0*ii)}

def get_grad(x, prior):
    a = x.clone().detach().requires_grad_(True)
    for f in prior.keys():
        z = prior[f]
        z = torch.sum(z(a[0][f]), dim=0)
        z.backward()
    return a.grad

def get_grads_to_match(ip, prior):
    return get_grad(ip, prior)

In [24]:
for ensemble in range(ensemble_size):
    loss_func = nn.CrossEntropyLoss()
    credo_model = Model(7,sample_size=len(data))
    optimizer = optim.Adam([{'params': credo_model.parameters()}], lr = 0.001, weight_decay=1e-4)

    for ep in range(0,epoch): 
        trainval = DataSet(X_train,y_train)
        train_loader = DataLoader(trainval, batch_size=batch_size)
        for input_data, target in train_loader:
            credo_model.zero_grad()
            input_data.requires_grad=True
            output = credo_model(input_data)
            
            calc_grads = (autograd.grad(torch.sum(output[0], dim=0), input_data, retain_graph=True, create_graph=True)[0])
            grads_to_match = get_grads_to_match(input_data, prior) 
            hinge_input = torch.abs(grads_to_match - calc_grads)
            loss = loss_func(output,target.squeeze()) + 0.01 * torch.norm(torch.clamp(hinge_input, min=0), p=1)
            
            loss.backward()
            optimizer.step()
        if ep%interval == 0:
            print(ep, interval)
            val = DataSet(X_val,y_val)
            val_loader = DataLoader(val, batch_size=1)
            acc_val = 0
            acc_test = 0
            
            for input_data, target in val_loader:
                output = credo_model(input_data)
                acc = multi_acc(output, target.unsqueeze(1))
                acc_val += acc

            print ('validation accuracy:', float(acc_val/len(val_loader)))
            testval = DataSet(X_test,y_test)
            test_loader = DataLoader(testval, batch_size=1)
            
            for input_data, target in test_loader:
                output = credo_model(input_data)
                acc = multi_acc(output, target.unsqueeze(1))
                acc_test += acc
                
            print('test accuracy:', float(acc_test/len(test_loader)))
            print()
    print("************")
    torch.save(credo_model, "models/credo_sachs_"+str(ensemble+1))

0 5
validation accuracy: 60.79999923706055
test accuracy: 59.599998474121094

5 5
validation accuracy: 78.5
test accuracy: 80.0

10 5
validation accuracy: 78.5
test accuracy: 80.9000015258789

15 5
validation accuracy: 79.19999694824219
test accuracy: 81.4000015258789

20 5
validation accuracy: 79.30000305175781
test accuracy: 81.69999694824219

25 5
validation accuracy: 79.30000305175781
test accuracy: 81.69999694824219

30 5
validation accuracy: 79.30000305175781
test accuracy: 81.69999694824219

35 5
validation accuracy: 79.30000305175781
test accuracy: 81.69999694824219

40 5
validation accuracy: 79.30000305175781
test accuracy: 81.69999694824219

45 5
validation accuracy: 79.30000305175781
test accuracy: 81.69999694824219

************
0 5
validation accuracy: 60.79999923706055
test accuracy: 59.599998474121094

5 5
validation accuracy: 60.79999923706055
test accuracy: 59.599998474121094

10 5
validation accuracy: 78.5
test accuracy: 81.19999694824219

15 5
validation accuracy: 79

In [25]:
n_classes=3
num_feat=7
num_alpha=3

cov = np.cov(X_values, rowvar=False)
mean_vector = np.mean(X_values, axis=0)

aces_credo_total = []

for ensemble in range(ensemble_size):
    ace_credo_total = []
    model = torch.load("models/credo_sachs_"+str(ensemble+1))
    for output_index in range(0,1):
        for t in range(0,num_feat):
            expectation_do_x = []
            inp=copy.deepcopy(mean_vector)
            for x in np.linspace(0, 1, num_alpha):                
                inp[t] = x
                input_torchvar = autograd.Variable(torch.FloatTensor(inp), requires_grad=True)
                output=model(input_torchvar)
                o1=output.data.cpu()
                val=o1.numpy()[output_index]#first term in interventional expectation                                       

                grad_mask_gradient = torch.zeros(n_classes)
                grad_mask_gradient[output_index] = 1.0
                #calculating the hessian
                first_grads = torch.autograd.grad(output.cpu(), input_torchvar.cpu(), grad_outputs=grad_mask_gradient, retain_graph=True, create_graph=True)

                for dimension in range(0,num_feat):#Tr(Hessian*Covariance)
                    if dimension == t:
                        continue
                    temp_cov = copy.deepcopy(cov)
                    temp_cov[dimension][t] = 0.0#row,col in covariance corresponding to the intervened one made 0
                    grad_mask_hessian = torch.zeros(num_feat)
                    grad_mask_hessian[dimension] = 1.0

                    #calculating the hessian
                    hessian = torch.autograd.grad(first_grads, input_torchvar, grad_outputs=grad_mask_hessian, retain_graph=True, create_graph=False)
                    val += np.sum(0.5*hessian[0].data.numpy()*temp_cov[dimension])#adding second term in interventional expectation
                expectation_do_x.append(val)#append interventional expectation for given interventional value
            ace_credo_total.append(np.array(expectation_do_x) - np.mean(np.array(expectation_do_x)))
    aces_credo_total.append(ace_ca_total)
np.save('./aces/sachs_credo_total.npy',aces_credo_total,allow_pickle=True)

In [26]:
rmse_results = []
frechet_results = []

for ensemble in range(ensemble_size):
    rmse_results.append([rmse(aces_gt[0], aces_credo_total[ensemble][0]),
                         rmse(aces_gt[1], aces_credo_total[ensemble][1]),
                         rmse(aces_gt[2], aces_credo_total[ensemble][2]),
                         rmse(aces_gt[3], aces_credo_total[ensemble][3]),
                         rmse(aces_gt[4], aces_credo_total[ensemble][4]),
                         rmse(aces_gt[5], aces_credo_total[ensemble][5]),
                         rmse(aces_gt[6], aces_credo_total[ensemble][6])])
    
    frechet_results.append([frechet_dist(aces_gt[0].reshape(3,1), aces_credo_total[ensemble][0].reshape(3,1)),
                            frechet_dist(aces_gt[1].reshape(3,1), aces_credo_total[ensemble][1].reshape(3,1)),
                            frechet_dist(aces_gt[2].reshape(3,1), aces_credo_total[ensemble][2].reshape(3,1)),
                            frechet_dist(aces_gt[3].reshape(3,1), aces_credo_total[ensemble][3].reshape(3,1)),
                            frechet_dist(aces_gt[4].reshape(3,1), aces_credo_total[ensemble][4].reshape(3,1)),
                            frechet_dist(aces_gt[5].reshape(3,1), aces_credo_total[ensemble][5].reshape(3,1)),
                            frechet_dist(aces_gt[6].reshape(3,1), aces_credo_total[ensemble][6].reshape(3,1))])
    
rmse_results = np.array(rmse_results)
print("rmse mean: ", np.mean(rmse_results, axis=0))
print("rmse std: ", np.std(rmse_results, axis=0))
print("rmse all features mean: ", np.mean(np.mean(rmse_results, axis=0)))
print("rmse all features std: ", np.mean(np.std(rmse_results, axis=0)))

print("frechet mean: ", np.mean(frechet_results, axis=0))
print("frechet std: ", np.std(frechet_results, axis=0))
print("frechet all features mean: ", np.mean(np.mean(frechet_results, axis=0)))
print("frechet all features std: ", np.mean(np.std(frechet_results, axis=0)))

rmse mean:  [0.08285197 3.81488683 0.02983257 0.42860937 2.87066199 0.13646983
 0.04895687]
rmse std:  [0.00000000e+00 4.44089210e-16 0.00000000e+00 5.55111512e-17
 0.00000000e+00 0.00000000e+00 0.00000000e+00]
rmse all features mean:  1.0588956327825614
rmse all features std:  7.137148015447435e-17
frechet mean:  [0.11701838 5.02455987 0.03905235 0.56964442 4.04197361 0.1671408
 0.0688235 ]
frechet std:  [0. 0. 0. 0. 0. 0. 0.]
frechet all features mean:  1.4326018477303641
frechet all features std:  0.0


# AHCE

In [58]:
for ensemble in range(ensemble_size):
    # Interval / Epochs
    
    mse_loss_func = nn.MSELoss()
    loss_func = nn.CrossEntropyLoss()
    ahce_model = Model(7,sample_size=len(data))

    optimizer = optim.Adam([{'params': ahce_model.parameters()}], lr = 0.001, weight_decay=1e-4)

    for ep in range(0,100):
        trainval = DataSet(X_train,y_train)
        train_loader = DataLoader(trainval, batch_size=64)
        for input_data, target in train_loader: 
            for phase in ['train_dag', 'freeze']:
                ahce_model.zero_grad()
                if phase == 'freeze':
                    output = ahce_model(input_data)
                    loss = loss_func(output,target.squeeze())
                    loss.backward(retain_graph=True)

                else:
                    output, pka_sample, raf_sample, mek_sample, erk_sample, jnk_sample, p38_sample  = ahce_model(input_data, phase='train_dag')             
                    
                    loss = 0.01*loss_func(output,target.squeeze()) 
                    loss.backward(retain_graph=True)
                optimizer.step()

        if ep%interval == 0:
            print(ep, interval)
            val = DataSet(X_val,y_val)
            val_loader = DataLoader(val, batch_size=1)
            acc_val = 0
            acc_test = 0

            for input_data, target in val_loader:
                output = ahce_model(input_data)
                acc = multi_acc(output, target.unsqueeze(1))
                acc_val += acc

            print ('validation accuracy:', float(acc_val/len(val_loader)))
            testval = DataSet(X_test,y_test)
            test_loader = DataLoader(testval, batch_size=1)

            for input_data, target in test_loader:
                output = ahce_model(input_data)
                acc = multi_acc(output, target.unsqueeze(1))
                acc_test += acc

            print('test accuracy:', float(acc_test/len(test_loader)))
            print()

    torch.save(ahce_model, "./models/ahce_sachs_"+str(ensemble+1))

0 5
validation accuracy: 60.20000076293945
test accuracy: 61.599998474121094

5 5
validation accuracy: 81.5999984741211
test accuracy: 80.3499984741211

10 5
validation accuracy: 82.19999694824219
test accuracy: 81.25

15 5
validation accuracy: 82.5999984741211
test accuracy: 81.44999694824219

20 5
validation accuracy: 82.5999984741211
test accuracy: 81.44999694824219

25 5
validation accuracy: 82.5999984741211
test accuracy: 81.44999694824219

30 5
validation accuracy: 82.5999984741211
test accuracy: 81.44999694824219

35 5
validation accuracy: 82.5999984741211
test accuracy: 81.44999694824219

40 5
validation accuracy: 82.5999984741211
test accuracy: 81.44999694824219

45 5
validation accuracy: 82.5999984741211
test accuracy: 81.44999694824219

50 5
validation accuracy: 82.5999984741211
test accuracy: 81.44999694824219

55 5
validation accuracy: 82.5999984741211
test accuracy: 81.44999694824219

60 5
validation accuracy: 82.5999984741211
test accuracy: 81.44999694824219

65 5
valida

In [59]:
n_classes=3
num_c=7#no. of features
num_alpha=3

aces_ahce_total = []
for ensemble in range(5):
    ace_ahce_total = []
    model =  torch.load("./models/ahce_sachs_"+str(ensemble+1))
    for output_index in range(1,2):#For every class
        #plt.figure()
        for t in range(0,num_c):#For every feature
            expectation_do_x = []
            for x in [0,0.5,1]:
                X_values[:,t] = x
                sample_data = model(X_values, phase='sample', inde=t, alpha=x).detach().numpy()
                cov = np.cov(sample_data, rowvar=False)
                means = np.mean(sample_data, axis=0)
                cov=np.array(cov)
                mean_vector = np.array(means)
                inp=copy.deepcopy(mean_vector)
                inp[t] = x
                input_torchvar = autograd.Variable(torch.FloatTensor(inp), requires_grad=True)

                output=model(input_torchvar)

                o1=output.data.cpu()
                val=o1.numpy()[output_index]#first term in interventional expectation                                       

                grad_mask_gradient = torch.zeros(n_classes)
                grad_mask_gradient[output_index] = 1.0
                #calculating the hessian
                first_grads = torch.autograd.grad(output.cpu(), input_torchvar.cpu(), grad_outputs=grad_mask_gradient, retain_graph=True, create_graph=True)

                for dimension in range(0,num_c):#Tr(Hessian*Covariance)
                    if dimension == t:
                        continue
                    temp_cov = copy.deepcopy(cov)
                    temp_cov[dimension][t] = 0.0#row,col in covariance corresponding to the intervened one made 0
                    grad_mask_hessian = torch.zeros(num_c)
                    grad_mask_hessian[dimension] = 1.0

                    #calculating the hessian
                    hessian = torch.autograd.grad(first_grads, input_torchvar, grad_outputs=grad_mask_hessian, retain_graph=True, create_graph=False)
                    val += np.sum(0.5*hessian[0].data.numpy()*temp_cov[dimension])#adding second term in interventional expectation
                expectation_do_x.append(val)#append interventional expectation for given interventional value

            ace_ahce_total.append(np.array(expectation_do_x) - np.mean(np.array(expectation_do_x)))

    aces_ahce_total.append(ace_ahce_total)
np.save('./aces/sachs_ahce_total.npy',aces_ahce_total,allow_pickle=True)

/tmp/ipykernel_1986/930309753.py:54: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pka_sample = self.causal_link_pkc_pka(torch.tensor(pkc_sample, dtype=torch.float))
/tmp/ipykernel_1986/930309753.py:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  raf_sample = self.causal_link_pkc_pka_raf(torch.cat((torch.tensor(pkc_sample, dtype=torch.float), pka_sample), dim=1))
/tmp/ipykernel_1986/930309753.py:56: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  mek_sample = self.causal_link_pkc_pka_raf_mek(torch.cat((torch.tensor(pkc_sample, dtype=torch

In [60]:
rmse_results = []
frechet_results = []

for ensemble in [0,1,2,3,4]:
    rmse_results.append([rmse(aces_gt[0], aces_ahce_total[ensemble][0]),
                         rmse(aces_gt[1], aces_ahce_total[ensemble][1]),
                         rmse(aces_gt[2], aces_ahce_total[ensemble][2]),
                         rmse(aces_gt[3], aces_ahce_total[ensemble][3]),
                         rmse(aces_gt[4], aces_ahce_total[ensemble][4]),
                         rmse(aces_gt[5], aces_ahce_total[ensemble][5]),
                         rmse(aces_gt[6], aces_ahce_total[ensemble][6])])
    
    frechet_results.append([frechet_dist(aces_gt[0].reshape(3,1), aces_ahce_total[ensemble][0].reshape(3,1)),
                            frechet_dist(aces_gt[1].reshape(3,1), aces_ahce_total[ensemble][1].reshape(3,1)),
                            frechet_dist(aces_gt[2].reshape(3,1), aces_ahce_total[ensemble][2].reshape(3,1)),
                            frechet_dist(aces_gt[3].reshape(3,1), aces_ahce_total[ensemble][3].reshape(3,1)),
                            frechet_dist(aces_gt[4].reshape(3,1), aces_ahce_total[ensemble][4].reshape(3,1)),
                            frechet_dist(aces_gt[5].reshape(3,1), aces_ahce_total[ensemble][5].reshape(3,1)),
                            frechet_dist(aces_gt[6].reshape(3,1), aces_ahce_total[ensemble][6].reshape(3,1))])
    
rmse_results = np.array(rmse_results)
print("rmse mean: ", np.mean(rmse_results, axis=0))
print("rmse std: ", np.std(rmse_results, axis=0))
print("rmse all features mean: ", np.mean(np.mean(rmse_results, axis=0)))
print("rmse all features std: ", np.mean(np.std(rmse_results, axis=0)))

print("frechet mean: ", np.mean(frechet_results, axis=0))
print("frechet std: ", np.std(frechet_results, axis=0))
print("frechet all features mean: ", np.mean(np.mean(frechet_results, axis=0)))
print("frechet all features std: ", np.mean(np.std(frechet_results, axis=0)))

rmse mean:  [0.12475822 0.65547383 0.12277304 0.14353915 0.51898169 0.0186962
 0.02440762]
rmse std:  [0.06767317 0.1756164  0.03766031 0.01556702 0.34269087 0.01361813
 0.01961943]
rmse all features mean:  0.2298042509487662
rmse all features std:  0.0960636188516346
frechet mean:  [0.171739   0.91099781 0.1714031  0.17648882 0.70143739 0.02320759
 0.02991145]
frechet std:  [0.09771076 0.23724553 0.05571379 0.01884757 0.45834208 0.01672321
 0.0240606 ]
frechet all features mean:  0.3121693101549147
frechet all features std:  0.12980622024131902
